Mounted at /content/drive


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option("display.max_columns", 200)
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
# Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
# Redes Neurais
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from sklearn.metrics import classification_report 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder




In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier

In [4]:
path  = '' # COLOCAR O CAMINHO CORRETO DOS ARQUIVOS ABAIXO

In [5]:
df = pd.read_csv(path + 'attack_dataset.csv.gz') # attack 
bonafide = pd.read_csv(path + 'bonafide_dataset_20191121.csv.gz') # bonafide traffic from mawilab
bonafide = pd.concat([bonafide, pd.read_csv(path + 'bonafide_dataset_20201110.csv.gz')])
bonafide = pd.concat([bonafide, pd.read_csv(path+ 'bonafide_dataset_20201129.csv.gz')])
print(df.shape, bonafide.shape)



(455503, 42) (380438, 41)


In [6]:
df.head(1)

,frame_info.encap_type,frame_info.time,frame_info.time_epoch,frame_info.number,frame_info.len,frame_info.cap_len,eth.type,ip.version,ip.hdr_len,ip.tos,ip.id,ip.flags,ip.flags.rb,ip.flags.df,ip.flags.mf,ip.frag_offset,ip.ttl,ip.proto,ip.checksum,ip.src,ip.dst,ip.len,ip.dsfield,tcp.srcport,tcp.dstport,tcp.seq,tcp.ack,tcp.len,tcp.hdr_len,tcp.flags,tcp.flags.fin,tcp.flags.syn,tcp.flags.reset,tcp.flags.push,tcp.flags.ack,tcp.flags.urg,tcp.flags.cwr,tcp.window_size,tcp.checksum,tcp.urgent_pointer,tcp.options.mss_val,label
0,1,"Sep 2, 2020 21:04:37.063530000 -03",1.599091e+09,1,74,74,0x00000800,4,20,NaN,0x00001b58,0x00000000,0,0,0,0,35,6,0x00003e78,179.241.213.24,172.26.8.200,60,0x00000000,1697,5802,0,0,0,40,0x00000002,0,1,0,0,0,0,0,64240,0x0000c9e9,0,1410.0,nmap_null


In [7]:
bonafide.head(1)

,frame_info.encap_type,frame_info.time,frame_info.time_epoch,frame_info.number,frame_info.len,frame_info.cap_len,eth.type,ip.version,ip.hdr_len,ip.tos,ip.id,ip.flags,ip.flags.rb,ip.flags.df,ip.flags.mf,ip.frag_offset,ip.ttl,ip.proto,ip.checksum,ip.src,ip.dst,ip.len,ip.dsfield,tcp.srcport,tcp.dstport,tcp.seq,tcp.ack,tcp.len,tcp.hdr_len,tcp.flags,tcp.flags.fin,tcp.flags.syn,tcp.flags.reset,tcp.flags.push,tcp.flags.ack,tcp.flags.urg,tcp.flags.cwr,tcp.window_size,tcp.checksum,tcp.urgent_pointer,tcp.options.mss_val
0,1,"Nov 21, 2019 02:00:00.309420000 -03",1.574312e+09,2,1474,74,0x000086dd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,443.0,51883.0,1.0,1.0,1400.0,20.0,0x00000010,0.0,0.0,0.0,0.0,1.0,0.0,0.0,131.0,0x0000308f,0.0,NaN


In [8]:
bonafide['label'] = "bonafide" # label column in the bonafide dataset


In [9]:
if (df.columns == bonafide.columns).all():
    examples_malicious = df.shape[0]
    examples_legitim = bonafide.shape[0]
    total = examples_malicious+examples_legitim
    print('Total examples of {0} with {1:0.2f} of attack and {2:0.2f} bonafide packets'.format(total, examples_malicious/total, examples_legitim/total))

Total examples of 835941 with 0.54 of attack and 0.46 bonafide packets


In [10]:
df.label.value_counts()


zmap             74613
nmap_connect     45882
hping_syn        43750
unicorn_syn      43039
nmap_syn         40642
unicorn_conn     39170
masscan          21138
nmap_ack         20497
nmap_window      18851
nmap_null        12511
nmap_xmas        12505
nmap_fin         12504
hping_ack        11344
nmap_maimon      10493
unicorn_ack       8494
hping_fin         7344
hping_null        7344
hping_xmas        7344
unicorn_null      4690
unicorn_xmas      4466
unicorn_fxmas     4444
unicorn_fin       4438
Name: label, dtype: int64

Pré-processamento

In [11]:
fields = ['eth.type', 'ip.id', 'ip.flags', 'ip.checksum', 'ip.dsfield', 'tcp.flags', 'tcp.checksum']

for field in fields:
    df[field] = df[field].apply(lambda x: int(str(x), 16))
    

In [12]:
bonafide = bonafide.fillna(0)
for field in fields:
    bonafide[field] = bonafide[field].apply(lambda x: int(str(x), 16))

In [13]:
full_data = pd.concat([bonafide, df])

In [14]:
wrong_proto = full_data[full_data['ip.proto'] != 6]['label'].value_counts().values
full_data = full_data[full_data['ip.proto'] == 6]
print("Found and removed", wrong_proto,"packets from the original dataset.")

Found and removed [52177] packets from the original dataset.


In [15]:
full_data.label.value_counts()


bonafide         328261
zmap              74613
nmap_connect      45882
hping_syn         43750
unicorn_syn       43039
nmap_syn          40642
unicorn_conn      39170
masscan           21138
nmap_ack          20497
nmap_window       18851
nmap_null         12511
nmap_xmas         12505
nmap_fin          12504
hping_ack         11344
nmap_maimon       10493
unicorn_ack        8494
hping_fin          7344
hping_null         7344
hping_xmas         7344
unicorn_null       4690
unicorn_xmas       4466
unicorn_fxmas      4444
unicorn_fin        4438
Name: label, dtype: int64

In [16]:
full_data.drop(columns=['frame_info.time', 'frame_info.encap_type', 'frame_info.time_epoch', 'frame_info.number', 
                        'frame_info.len', 'frame_info.cap_len', 'eth.type', 'ip.flags', 'ip.src', 'ip.dst',
                        'ip.version', 'ip.proto', 'tcp.flags'], axis=1, inplace=True)

In [17]:
full_data.describe().T

,count,mean,std,min,25%,50%,75%,max
ip.hdr_len,783764.0,2.000000e+01,0.000000e+00,20.0,20.0,20.0,20.0,2.000000e+01
ip.tos,328261.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00
ip.id,783764.0,3.020374e+04,2.037583e+04,0.0,11779.0,29827.0,48693.0,6.553500e+04
ip.flags.rb,783764.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00
ip.flags.df,783764.0,6.453920e-01,4.783947e-01,0.0,0.0,1.0,1.0,1.000000e+00
ip.flags.mf,783764.0,2.551789e-06,1.597431e-03,0.0,0.0,0.0,0.0,1.000000e+00
ip.frag_offset,783764.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.000000e+00
ip.ttl,783764.0,7.956949e+01,7.830542e+01,1.0,34.0,41.0,61.0,2.540000e+02
ip.checksum,783764.0,3.278110e+04,1.870863e+04,0.0,17178.0,32137.0,49056.0,6.553400e+04
ip.len,783764.0,1.802764e+02,4.113036e+02,40.0,40.0,52.0,60.0,8.289000e+03


In [18]:
# check features with zero variance, they do not support the learning task
(full_data.var() == 0)

ip.hdr_len              True
ip.tos                  True
ip.id                  False
ip.flags.rb             True
ip.flags.df            False
ip.flags.mf            False
ip.frag_offset          True
ip.ttl                 False
ip.checksum            False
ip.len                 False
ip.dsfield             False
tcp.srcport            False
tcp.dstport            False
tcp.seq                False
tcp.ack                False
tcp.len                False
tcp.hdr_len            False
tcp.flags.fin          False
tcp.flags.syn          False
tcp.flags.reset        False
tcp.flags.push         False
tcp.flags.ack          False
tcp.flags.urg          False
tcp.flags.cwr          False
tcp.window_size        False
tcp.checksum           False
tcp.urgent_pointer     False
tcp.options.mss_val    False
dtype: bool

Remover as colunas com variância zero

In [19]:
full_data.drop(columns=['ip.hdr_len', 'ip.tos', 'ip.flags.rb', 
                        'ip.flags.mf', 'ip.frag_offset'], axis=1, inplace=True)

In [20]:
full_data.columns


Index(['ip.id', 'ip.flags.df', 'ip.ttl', 'ip.checksum', 'ip.len', 'ip.dsfield',
       'tcp.srcport', 'tcp.dstport', 'tcp.seq', 'tcp.ack', 'tcp.len',
       'tcp.hdr_len', 'tcp.flags.fin', 'tcp.flags.syn', 'tcp.flags.reset',
       'tcp.flags.push', 'tcp.flags.ack', 'tcp.flags.urg', 'tcp.flags.cwr',
       'tcp.window_size', 'tcp.checksum', 'tcp.urgent_pointer',
       'tcp.options.mss_val', 'label'],
      dtype='object')

Outras colunas removidas

In [21]:
full_data.drop(columns=["ip.checksum", "tcp.checksum", 
                        "tcp.ack", "tcp.dstport"], axis=1, inplace=True)

Retirada de duplicatas

In [22]:
full_data.drop_duplicates(inplace=True, ignore_index=True)


Classificação multiclasse


In [23]:
full_data['label'].value_counts()


bonafide         315729
zmap              70498
nmap_connect      45358
hping_syn         43260
unicorn_syn       42479
nmap_syn          40112
unicorn_conn      38629
masscan           21114
nmap_ack          20002
nmap_window       18359
nmap_null         12018
nmap_fin          12013
nmap_xmas         12012
hping_ack         10877
nmap_maimon       10003
unicorn_ack        8023
hping_fin          6877
hping_null         6877
hping_xmas         6877
unicorn_null       4219
unicorn_xmas       3995
unicorn_fxmas      3973
unicorn_fin        3967
Name: label, dtype: int64

In [24]:
full_data.shape[0]

757271

In [26]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier

algorithms = {
    #"MLP" : (MLPClassifier(), {
    #    "hidden_layer_sizes" : (10, 10),
    #}),
    #"XGB" : (XGBClassifier(), {}),
    # "NB" : (GaussianNB(), {}),
    # "LR" : (LogisticRegression(n_jobs=-1), {
    #     "class_weight": [None, "balanced"]
    # }),
    "RF" : (RandomForestClassifier(random_state=17, n_jobs=-1), {
        "n_estimators" : [10, 50, 100, 200],
        "criterion" : ("gini", "entropy"), 
        "max_depth": [5, 10],
        "class_weight": (None, "balanced", "balanced_subsample")
    }),
    # "DT" : (DecisionTreeClassifier(), {
    #     "criterion": ("gini", "entropy"), 
    #     "max_depth": [x for x in range(1,21)],
    #     "class_weight": (None, "balanced")
    # }),
}
le = LabelEncoder()
full_data = full_data.fillna(0)
X = full_data.drop(columns = ["label"])
full_data['label'] = le.fit_transform(full_data.label)
y = full_data.label

print(X.shape, y.shape)

(757271, 19) (757271,)


In [27]:
X = X.astype(int) 
X.head()

,ip.id,ip.flags.df,ip.ttl,ip.len,ip.dsfield,tcp.srcport,tcp.seq,tcp.len,tcp.hdr_len,tcp.flags.fin,tcp.flags.syn,tcp.flags.reset,tcp.flags.push,tcp.flags.ack,tcp.flags.urg,tcp.flags.cwr,tcp.window_size,tcp.urgent_pointer,tcp.options.mss_val
0,46834,0,247,40,0,53026,0,0,20,0,1,0,0,0,0,0,1024,0,0
1,3793,1,56,537,8,443,1,485,32,0,0,0,1,1,0,0,252,0,0
2,0,1,59,80,0,59206,1,0,60,0,0,0,0,1,0,0,4677,0,0
3,8559,1,55,54,0,52239,1,2,32,0,0,0,1,1,0,0,115,0,0
4,54321,0,244,40,8,44034,0,0,20,0,1,0,0,0,0,0,65535,0,0


In [28]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, make_scorer, f1_score, roc_auc_score

kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=17) # Train, Test
gskf = StratifiedKFold(n_splits=3, shuffle=True, random_state=17) # Validation
perf = f1_score # could be considered roc_auc_score for binary classification
acc = accuracy_score

score = {}
accuracy = {}
best_params = {}
results = {}
for algorithm in algorithms.keys():
    score[algorithm] = []
    accuracy[algorithm] = []
    results[algorithm] = { 'expected' : [], 'predicted' : []}

for algorithm, (clf, parameters) in algorithms.items():
    print(algorithm)
    for train, test in kf.split(X, y):
        prep = StandardScaler()
        #prep = MinMaxScaler()
        prep.fit(X.iloc[train])
        best = GridSearchCV(clf, parameters, cv=gskf, scoring=make_scorer(perf), n_jobs=-1)
        best.fit(prep.transform(X.iloc[train]), y.iloc[train])
        best_params[algorithm] = best.best_params_
        print(best.best_params_)
        score[algorithm].append(perf(best.predict(prep.transform(X.iloc[test])), y.iloc[test], average='weighted'))
        accuracy[algorithm].append(acc(best.predict(prep.transform(X.iloc[test])), y.iloc[test]))
        results[algorithm]['expected'].extend(y.iloc[test])
        results[algorithm]['predicted'].extend(best.predict_proba(prep.transform(X.iloc[test])).transpose()[1]) 


RF
{'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'n_estimators': 10}
{'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'n_estimators': 10}
{'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'n_estimators': 10}
{'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'n_estimators': 10}
{'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'n_estimators': 10}
{'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'n_estimators': 10}
{'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'n_estimators': 10}
{'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'n_estimators': 10}
{'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'n_estimators': 10}
{'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'n_estimators': 10}


In [29]:
count = 'rf_'

In [30]:
# f1-scores for the best set of parameters from the Grid Search above (for each k-fold)
scores_df = pd.DataFrame.from_dict(score)
scores_df

,RF
0,0.620089
1,0.618702
2,0.621180
3,0.621718
4,0.620029
5,0.622118
6,0.620596
7,0.620223
8,0.622523
9,0.618952


In [31]:
scores_df.to_csv(path+count+'scores.csv', index=False)

In [32]:
best_params_ = pd.DataFrame.from_dict(best_params)
best_params_

,RF
class_weight,None
criterion,gini
max_depth,5
n_estimators,10


In [33]:
best_params_.to_csv(path+count+'best_models.csv', index=False)


In [34]:
acc_df = pd.DataFrame.from_dict(accuracy)
acc_df

,RF
0,0.558010
1,0.558638
2,0.558084
3,0.555363
4,0.555429
5,0.558810
6,0.557318
7,0.556301
8,0.556341
9,0.553224


In [35]:
acc_df.to_csv(path+count+'acc.csv', index=False)

In [36]:
auc_df = pd.DataFrame.from_dict(results)
auc_df

,RF
expected,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
predicted,"[0.001662415994936426, 0.005550670703229746, 0..."


In [37]:
auc_df.to_csv(path+count+'auc.csv', index=False)

In [ ]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, make_scorer, f1_score, roc_auc_score, roc_curve

kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=17) # Train, Test
gskf = StratifiedKFold(n_splits=3, shuffle=True, random_state=17) # Validation
perf = roc_auc_score

results = {}

for algorithm in algorithms.keys():
    results[algorithm] = { 'expected' : [], 'predicted' : []}
    
for algorithm, (clf, parameters) in algorithms.items():
    print(algorithm)
    for train, test in kf.split(X, y):
        prep = StandardScaler()
        prep.fit(X.iloc[train])

        best = GridSearchCV(clf, parameters, cv=gskf, scoring=make_scorer(perf), n_jobs=-1)
        best.fit(prep.transform(X.iloc[train]), y.iloc[train])
        
        results[algorithm]['expected'].extend(y.iloc[test])
        results[algorithm]['predicted'].extend(best.predict_proba(prep.transform(X.iloc[test])).transpose()[1]) 